In [2]:
import os

# Add OpenAI library
import openai

# Get Configuration Settings
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
openai.__version__

'0.28.1'

In [4]:
# Configure OpenAI API using Azure OpenAI
openai.api_key = os.getenv("API_KEY")
openai.api_base = os.getenv("ENDPOINT")
openai.api_type = "azure"  # Necessary for using the OpenAI library with Azure OpenAI
openai.api_version = "2024-02-01"  # Latest / target version of the API

In [5]:
from langchain.embeddings import OpenAIEmbeddings

In [6]:
# OpenAI Settings
model_deployment = "text-embedding-ada-002"
# SDK calls this "engine", but naming it "deployment_name" for clarity

model_name = "text-embedding-ada-002"

In [7]:
openai_embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    openai_api_version = os.getenv("OPENAI_API_VERSION"), openai_api_key = os.getenv("API_KEY"),
    openai_api_base = os.getenv("ENDPOINT"), openai_api_type = "azure"
)

/var/folders/l7/m3wgz92x6vj1gkr9lwh0893c0000gn/T/ipykernel_880/3172091303.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  openai_embeddings: OpenAIEmbeddings = OpenAIEmbeddings(


In [28]:
from langchain_chroma import Chroma

vector_store_contoso = Chroma(
    collection_name="Contoso-Outdoor-Docs",
    embedding_function=openai_embeddings,
    persist_directory="../Contoso-Outdoor-Vector-DB",  # Where to save data locally, remove if not neccesary
)

## Index markdown files

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document

def load_and_process_markdowns(pdf_folder_path):
        """
        This method is responsible for upserting PDF content.
        It loads the PDF file, splits the content into chunks, and then upserts the chunks into VecDB.
        """
        documents = []
        for file in os.listdir(pdf_folder_path):
            if file.endswith(".md"):
                pdf_path = os.path.join(pdf_folder_path, file)
                loader = UnstructuredMarkdownLoader(pdf_path)
                documents.extend(loader.load())
        
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150, separators=["\n", " ", "?", ".", "!"])
        docs = text_splitter.split_documents(documents)
        return docs

In [23]:
markdown_folder_path =  "../data/manual_info"
splits = load_and_process_markdowns(markdown_folder_path)

In [27]:
print(len(splits))
splits[0]

643


Document(metadata={'source': '../data/manual_info/product_info_2.md'}, page_content='Information about product item_number: 2\n\nAdventurer Pro Backpack, price $90,\n\nBrand\n\nHikeMate\n\nCategory\n\nBackpacks\n\nFeatures\n\n40L capacity for ample storage space\n\nErgonomic design for comfortable carrying\n\nDurable nylon material for long-lasting performance\n\nMultiple compartments and pockets for organized storage\n\nHydration system compatibility with a dedicated hydration bladder sleeve and tube port\n\nAdjustable and padded shoulder straps for a customized fit and enhanced comfort')

In [29]:
import time
from uuid import uuid4

batch_size = 50  # Adjust this batch size based on your rate limit
delay = 60  # Delay in seconds between batches

for i in range(0, len(splits), batch_size):
    batch = splits[i:i+batch_size]
    uuids = [str(uuid4()) for _ in range(len(batch))]
    print(f"Upserting {i} documents")
    # try:
    response = vector_store_contoso.add_documents(documents=batch, ids=uuids)
    #     print(f"Response: {response}")
    # except Exception as e:
    #     print(e)
    time.sleep(delay)  # Delay to prevent hitting rate limits


Upserting 0 documents
Upserting 50 documents
Upserting 100 documents
Upserting 150 documents
Upserting 200 documents
Upserting 250 documents
Upserting 300 documents
Upserting 350 documents
Upserting 400 documents
Upserting 450 documents
Upserting 500 documents
Upserting 550 documents
Upserting 600 documents


## Index json files.